In [17]:
from math import floor
from PIL import Image
from scipy import ndimage
import numpy as np
import os
import os.path
import time
import random
import cv2


class Preprocess():
    def __init__(self, path, save=False, scale=1, amount=0.001, keep_percentage=0.1, lowThreshold=100, highThreshold=300, kernel_size=(5,5), iterations=1):
        self.path = self.get_path(path)
        self.func = self.get_func()
        self.scale = self.get_scale(scale)
        self.amount = self.get_amount(amount)
        self.save = self.get_save(save)
        self.keep_percentage = self.get_perc(keep_percentage)
        self.lowThreshold = self.get_low(lowThreshold)
        self.highThreshold = self.get_high(highThreshold)
        self.kernel_size = self.get_kernel(kernel_size)
        self.iterations = self.get_iter(iterations)
        self.func(self.path, self.save, self.scale, self.amount, self.keep_percentage, self.lowThreshold, self.highThreshold, self.kernel_size, self.iterations)

    def get_save(self, save):
        return save
    def get_scale(self, scale):
        return scale
    def get_amount(self, amount):
        return amount
    def get_perc(self, keep_percentage):
        return keep_percentage
    def get_low(self, lowThreshold):
        return lowThreshold
    def get_high(self, highThreshold):
        return highThreshold
    def get_kernel(self, kernel_size):
        return kernel_size
    def get_iter(self, iterations):
        return iterations
    
    def get_path(self, path):
        """Get whole path of an image

        If path does not start with '/', then try to open image from pwd
        If path starts with '/', then open image of given path
        """
        if path.startswith('/') and not path.startswith('~/'):
            return os.getcwd() + '/' + path
        else:
            return path

    def get_func(self):
        "Get dithering function to run"
        return self.algo
              
    def add_noise(self, image, amount):
        #Gets image array input and adds black speckles
        
        num_pepper = np.ceil(amount * image.size)
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
        image[tuple(coords)] = 0

        return image
    
    def remove_points(self, image, keep_percentage):
        
        # Remove points: input is an array, output is an array
        img = image
        img.tolist()
        xlim, ylim = len(img[0]), len(img)
        
        for y in range(0, ylim):
            for x in range(0, xlim):
                #Only alter black pixels
                if not img[y, x]:
                    #Keep 'percentage' amount of black pixels
                    if random.uniform(0, 1) >= keep_percentage:
                        img[y, x] = 255
        img = np.array(img) 
        
        return img
    
    def edge_detection(self, image, lowThreshold, highThreshold, kernel_size, iterations):
        #Canny edge detection
        img = cv2.Canny(image, lowThreshold, highThreshold)
        
        #Dilate edges to make them bigger
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
        img = cv2.dilate(img, kernel, iterations=iterations)
        
        # Inverse image
        img = cv2.bitwise_not(img)
        
        return img
    
    def process(self, image, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations):
        
        #Convert image to scipy array
        img = np.array(image)

        # Edge detection
        img = self.edge_detection(img, lowThreshold, highThreshold, kernel_size, iterations)
        
        #Remove points
        img = self.remove_points(img, keep_percentage)
        
        # Add Noise
        img = self.add_noise(img, amount)
        
        #Convert to PIL
        out = Image.fromarray(img)
        
        return out
    
    def save_image(self, image, image_file, scale, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations, width, height):
        """Saves (pillow) image to 'output' folder"""
        
        split_file_path = image_file.rsplit("\\", 2)
        save_path = split_file_path[0] +  "\\Outputs"
        
        folderName = save_path + "\\output" + time.strftime("%d%b%Y--%H-%M-%S")
        os.makedirs(folderName)
        
        #Save image
        image.save(folderName + "\\image.png")
                
        #Save settings
        settingsName = os.path.join(folderName, "settings.txt")         
        file = open(settingsName, "w")
        
        file.write("scale:{}\namount:{}\nkeep_percentage:{}\nlowThreshold:{}\nhighThreshold:{}\nkernel_size:{}\niterations:{}\n\nsize:{} x {}".format(scale, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations, width, height))
        
        #Save pixels
        completeName = os.path.join(folderName, "pixels.txt")         
        file = open(completeName, "w")
        
        file.write("[")
        
        image = np.array(image)
        image.tolist()
        xlim, ylim = len(image[0]), len(image)
        
        for y in range(0, ylim):
            for x in range(0, xlim):
                
                #Only output black pixels
                if not image[y, x]:
                    file.write("[{}, {}],\n".format(x, y))
                    
        file.write("];")        

    def algo(self, image_file, save, scale, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations):
        """Perform algorithm"""
        
        #Open image
        
        img = Image.open(image_file)
        
        #Resizing image
        
        width, height = img.size
        size = (floor(width/scale), floor(height/scale))
        img = img.resize(size)
        width, height = img.size
        
        # Convert to b&w
        img = img.convert('L')
        img = self.process(img, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations)

        #Floyd-Steinberg dithering
        #img = img.convert('1')

        #Show & save pixels from image
        img.show()
        if save:
            self.save_image(img, image_file, scale, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations, width, height)

def main():
    #Call dithering algorithm
    Preprocess(path, save, scale, amount, keep_percentage, lowThreshold, highThreshold, kernel_size, iterations)



In [18]:
Preprocess(r"C:\Users\micha\OneDrive - TU Eindhoven\00. Studie Relevant\2IMV10\Python\img\abe.jpg", save=True, scale=1, amount=0.0001, keep_percentage=0.01, lowThreshold=200, highThreshold=300, kernel_size=(5,5), iterations=5)